In [8]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "***"

In [11]:
from google.cloud import bigquery as bq
client = bq.Client()
print(client)

In [108]:
dataset_id = f"{client.project}.PLeague_Dataset"
table_id = f"{dataset_id}.Player_Data"
tschema = [
bq.SchemaField("Name", "STRING", mode="NULLABLE"),
bq.SchemaField("Num", "INTEGER", mode="NULLABLE"),
bq.SchemaField("Team", "STRING", mode="NULLABLE"),
bq.SchemaField("G", "INTEGER", mode="NULLABLE"),
bq.SchemaField("Time", "INTEGER", mode="NULLABLE"),
bq.SchemaField("Two_P_Get", "INTEGER", mode="NULLABLE"),
bq.SchemaField("Two_P_Make", "INTEGER", mode="NULLABLE"),
bq.SchemaField("Two_P_Percent", "DECIMAL", mode="NULLABLE"),
bq.SchemaField("Three_P_Get", "INTEGER", mode="NULLABLE"),
bq.SchemaField("Three_P_Make", "INTEGER", mode="NULLABLE"),
bq.SchemaField("Three_P_Percent", "DECIMAL", mode="NULLABLE"),
bq.SchemaField("FT_Get", "INTEGER", mode="NULLABLE"),
bq.SchemaField("FT_Make", "INTEGER", mode="NULLABLE"),
bq.SchemaField("FT_Percent", "DECIMAL", mode="NULLABLE"),
bq.SchemaField("PTS", "DECIMAL", mode="NULLABLE"),
bq.SchemaField("OREB", "DECIMAL", mode="NULLABLE"),
bq.SchemaField("DREB", "DECIMAL", mode="NULLABLE"),
bq.SchemaField("REB", "DECIMAL", mode="NULLABLE"),
bq.SchemaField("ASS", "DECIMAL", mode="NULLABLE"),
bq.SchemaField("STL", "DECIMAL", mode="NULLABLE"),
bq.SchemaField("BLK", "DECIMAL", mode="NULLABLE"),
bq.SchemaField("TO", "DECIMAL", mode="NULLABLE"),
bq.SchemaField("FO", "DECIMAL", mode="NULLABLE"),
]
dataset = bq.Dataset(dataset_id)
dataset.location = "asia-east1"
dataset = client.create_dataset(dataset)
table = bq.Table(table_id, schema=tschema)
table = client.create_table(table)  # API request
print(f"Created dataset {client.project}.{dataset.dataset_id}")
print(f"Created table {table.project}.{table.dataset_id}.{table.table_id}")

Created dataset bigquery001-361808.PLeague_Dataset
Created table bigquery001-361808.PLeague_Dataset.Player_Data


In [109]:
import requests
import bs4

In [110]:
url = "https://pleagueofficial.com/stat-player"
url_content = bs4.BeautifulSoup(requests.get(url).text,"lxml")
player_data = url_content.find('tbody').find_all('tr')
all_data = []
for player in player_data:
    player_data = []
    name = player.find('th').a.text
    if len(name) >= 2:
        player_data.append(name)
        datas = player.find_all('td')
        for data in datas:
            player_data.append(data.text)
    else:
        continue
    all_data.append(player_data)

In [111]:
import pandas as pd
import decimal
all_data_df = pd.DataFrame(all_data, columns = ['Name','Num','Team','G','Time','Two_P_Get','Two_P_Make','Two_P_Percent','Three_P_Get','Three_P_Make','Three_P_Percent','FT_Get','FT_Make','FT_Percent','PTS','OREB','DREB','REB','ASS','STL','BLK','TO','FO'])

In [112]:
all_data_df['Num'] = all_data_df['Num'].astype('int')
all_data_df['G'] = all_data_df['G'].astype('int')
all_data_df['Time'] = all_data_df['Time'].apply(lambda x:int(x.split(':')[0])*60 + int(x.split(':')[1]))
all_data_df['Time'] = all_data_df['Time'].astype('int')
all_data_df['Two_P_Get'] = all_data_df['Two_P_Get'].astype('int')
all_data_df['Two_P_Make'] = all_data_df['Two_P_Make'].astype('int')
all_data_df['Two_P_Percent'] = all_data_df['Two_P_Percent'].apply(lambda x:x.replace('%',''))
all_data_df['Two_P_Percent'] = all_data_df['Two_P_Percent'].astype(str).map(decimal.Decimal)
all_data_df['Three_P_Get'] = all_data_df['Three_P_Get'].astype('int')
all_data_df['Three_P_Make'] = all_data_df['Three_P_Make'].astype('int')
all_data_df['Three_P_Percent'] = all_data_df['Three_P_Percent'].apply(lambda x:x.replace('%',''))
all_data_df['Three_P_Percent'] = all_data_df['Three_P_Percent'].astype(str).map(decimal.Decimal)
all_data_df['FT_Get'] = all_data_df['FT_Get'].astype('int')
all_data_df['FT_Make'] = all_data_df['FT_Make'].astype('int')
all_data_df['FT_Percent'] = all_data_df['FT_Percent'].apply(lambda x:x.replace('%',''))
all_data_df['FT_Percent'] = all_data_df['FT_Percent'].astype(str).map(decimal.Decimal)
all_data_df['PTS'] = all_data_df['PTS'].astype(str).map(decimal.Decimal)
all_data_df['OREB'] = all_data_df['OREB'].astype(str).map(decimal.Decimal)
all_data_df['DREB'] = all_data_df['DREB'].astype(str).map(decimal.Decimal)
all_data_df['REB'] = all_data_df['REB'].astype(str).map(decimal.Decimal)
all_data_df['ASS'] = all_data_df['ASS'].astype(str).map(decimal.Decimal)
all_data_df['STL'] = all_data_df['STL'].astype(str).map(decimal.Decimal)
all_data_df['BLK'] = all_data_df['BLK'].astype(str).map(decimal.Decimal)
all_data_df['TO'] = all_data_df['TO'].astype(str).map(decimal.Decimal)
all_data_df['FO'] = all_data_df['FO'].astype(str).map(decimal.Decimal)

In [114]:
all_data_df

,Name,Num,Team,G,Time,Two_P_Get,Two_P_Make,Two_P_Percent,Three_P_Get,Three_P_Make,...,FT_Percent,PTS,OREB,DREB,REB,ASS,STL,BLK,TO,FO
0,鑀倫,19,高雄17直播鋼鐵人,9,2038,25,56,44.64,31,91,...,89.19,19.56,0.44,4.67,5.11,3.33,2.33,0.56,1.89,2.89
1,奧帝,11,新北國王,5,1931,14,29,48.28,12,40,...,93.33,15.6,1,8.2,9.2,2.4,0.8,2,2,3
2,特壞,6,新竹街口攻城獅,12,2487,70,133,52.63,25,86,...,70.97,21.58,1.83,5.92,7.75,3.5,1.58,0.33,1.92,1.67
3,楊敬敏,6,新北國王,18,1693,51,116,43.97,26,85,...,79.27,13.61,0.56,3.06,3.61,2.61,1,0,2.06,1.17
4,林俊吉,11,福爾摩沙台新夢想家,19,1800,57,138,41.3,38,111,...,85,14.68,1.11,3.11,4.21,3.63,1.05,0.05,2.21,2.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,鄭德維,12,高雄17直播鋼鐵人,2,108,0,0,0,1,1,...,0,1.5,0,0,0,0,0,0,0,0.5
95,陳又瑋,4,高雄17直播鋼鐵人,18,2250,51,114,44.74,19,72,...,81.82,9.83,1.17,4.89,6.06,6.33,1.28,0.44,2.89,1.78
96,王律翔,8,高雄17直播鋼鐵人,18,1454,18,54,33.33,37,90,...,80,9.06,0.72,1.17,1.89,1.06,0.61,0.17,1.39,2.11
97,洪志善,24,新北國王,10,852,2,7,28.57,11,23,...,0,3.7,0.1,1.3,1.4,2.4,0.3,0,0.8,1.2


In [115]:
dataset_id = "PLeague_Dataset"
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table("Player_Data")

query_delete = f"""
    delete * from `Player_Data`
"""
print(query_delete)
# query_delete
job = client.query(query_delete)
job.result()
print("Deleted!")


job = client.load_table_from_dataframe(all_data_df, table_ref, location="asia-east1")
job.result()  # Waits for table load to complete.
assert job.state == "DONE"